In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [3]:
import os
import math
from typing import List

from glob import glob

from src.utils.fs_utils import safe_parse_json

In [4]:
ICNET_DICE_EXPERIMENTS_PATH = "/home/ppeczek/Dokumenty/YetAnotherSegmentationExperiment/resources/experiments/final_experiment/ic_net_ce_dice/"
ICNET_CE_EXPERIMENTS_PATH = "/home/ppeczek/Dokumenty/YetAnotherSegmentationExperiment/resources/experiments/final_experiment/ic_net_ce/"
UNET_DICE_EXPERIMENTS_PATH = "/home/ppeczek/Dokumenty/YetAnotherSegmentationExperiment/resources/experiments/final_experiment/u_net_original_ce_dice/"
UNET_CE_EXPERIMENTS_PATH = "/home/ppeczek/Dokumenty/YetAnotherSegmentationExperiment/resources/experiments/final_experiment/u_net_original_ce/"

In [5]:
DICE_MARKER = "D"
CLASSES_ABBREVIATIONS = {
    "background": "bg",
    "box": "bx",
    "clamp": "cl",
    "drill": "dr",
    "duck": "du", 
    "adapter": "ad",
    "bottle": "bo"
}
METRICS_TYPES_ABBREVIATIONS = {
    "pixel_weighted": "pw",
    "examples_weighted": "ew"
}
METRICS_TYPES_ORDER = ["pixel_weighted", "examples_weighted"]
CLASS_ORDER = ["background", "box", "clamp", "drill", "duck", "adapter", "bottle"]

In [6]:
def generate_table_with_experiment_results(experiments_base_path: str) -> str:
    metrics_paths = glob(os.path.join(
        experiments_base_path, "*", "evaluation_metrics.json"
    ))
    metrics_paths = sorted(metrics_paths)
    metrics = [safe_parse_json(path) for path in metrics_paths]
    table = _prepare_header()
    table += _prepare_table_content(metrics_paths, metrics)
    table += _prepare_table_end() 
    return table

def _prepare_header() -> str:
    header = r"\begin{center} \tiny \renewcommand{\arraystretch}{1.5} " \
        r"\begin{tabular}{ |c|c|c|c|c|c|c|c|c||c|c|c|c|c|c|c|c| } \hline "
    header += "Split name &"
    for mt in METRICS_TYPES_ORDER:
        for c in CLASS_ORDER:
            header += f" ${DICE_MARKER}^{{{METRICS_TYPES_ABBREVIATIONS[mt]}}}_{{{CLASSES_ABBREVIATIONS[c]}}}$ &"
        header += f" ${DICE_MARKER}^{{{METRICS_TYPES_ABBREVIATIONS[mt]}}}$ & "
    header = header[:-2]
    header += "\\\\ [1.05ex] \hline "
    header += "\n"
    return header

def _prepare_table_content(metrics_paths: List[str], metrics: List[dict]) -> str:
    result = ""
    for path, metric in zip(metrics_paths, metrics):
        result += _prepare_content_row(path=path, metric=metric)
    return result

def _prepare_content_row(path: str, metric: dict) -> str:
    experiment_name = os.path.basename(os.path.dirname(path))
    e_name, e_number =  experiment_name.split("#")
    if "rand" in e_name:
        e_name = "random"
    else:
        e_name = "rot"
    experiment_name_latex = f"${e_name}_{e_number}$"
    row = f"{experiment_name_latex} & "
    for mt in METRICS_TYPES_ORDER:
        key = f"{mt}_class_metrics"
        for c in CLASS_ORDER:
            score = round(metric[key]["dice"][c][0], 2)
            row += f" {score} &"
        key = f"{mt}_mean_metrics"
        score =  round(metric[key]["dice"], 2)
        row += f" {score} & "
    row = row[:-2]
    row += "\\\\ \hline \n"
    return row

def _prepare_table_end() -> str:
    return r"\end{tabular} \end{center}"

In [13]:
icnet_table = generate_table_with_experiment_results(
    experiments_base_path=UNET_CE_EXPERIMENTS_PATH
)

In [14]:
with open("UNET_CE.txt", "w") as f:
    f.write(icnet_table)